In [1]:
embeding_matrix = load_pickle_file("../../new_disk/embeding_matrix")
word2index = load_pickle_file("../../new_disk/word2index")
index_sequences = load_pickle_file("../../new_disk/index_sequences.pkl")
labels = load_pickle_file("./labels")
SEQUENCE_LENGTH = 434
VOCAB_SIZE = len(embeding_matrix)
EMB_DIMENSION = 300
VALIDATION_SPLIT = 0.1
PAD_SEQ_VALUE = 3001066

NameError: name 'load_pickle_file' is not defined

In [4]:
import numpy as np
same_lenght_seq = [seq[:SEQUENCE_LENGTH].reshape(1,SEQUENCE_LENGTH) if len(seq) >= SEQUENCE_LENGTH 
                   else np.concatenate((seq,np.array([PAD_SEQ_VALUE] * (SEQUENCE_LENGTH - len(seq)))),axis = 0).reshape(1,SEQUENCE_LENGTH) 
                   for seq in index_sequences]
same_lenght_seq = np.concatenate(same_lenght_seq,axis=0)
labels = np.vstack(np.array([1,0]).reshape(1,2) if x == 0 else np.array([0,1]).reshape(1,2) for x in labels)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  


Dataset split

In [5]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(same_lenght_seq, labels, test_size=0.2, random_state=42,shuffle = True)

In [6]:
data_size = len(x_train)
perm = np.random.permutation(data_size)
idx_train = perm[:int(data_size*(1-VALIDATION_SPLIT))]
idx_val = perm[int(data_size*(1-VALIDATION_SPLIT)):]

data_train = x_train[idx_train]
labels_train = y_train[idx_train]

data_val = x_train[idx_val]
labels_val = y_train[idx_val]

In [13]:
data_train.shape

(7380, 434)

<h1>LSTM</h1>

In [7]:
import tensorflow as tf
from tensorflow import keras as tk
from tensorflow.keras import layers as tfkl
from sklearn.metrics import mean_squared_error,mean_absolute_error

from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, TimeDistributed, Dropout, Activation, Flatten, Conv1D, MaxPool1D, SpatialDropout1D, GlobalMaxPool1D, BatchNormalization
from tensorflow.keras.models import Model
# from tensorflow.keras.layers.normalization import BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or 

In [8]:
model_checkpoint = ModelCheckpoint("./Checkpoints", save_best_only=True, save_weights_only=True)


In [8]:
NUM_FILTERS = 14
KERNEL_SIZE = 12
HIDDEN_UNITS = 4
rate_drop_lstm = 0.15
rate_drop_lstm = 0.15
LSTM_HIDDEN_DIM_SIZE = 5
model = tk.Sequential()
embedding_layer = Embedding(VOCAB_SIZE,
        EMB_DIMENSION,
        weights=[embeding_matrix],
        input_length=SEQUENCE_LENGTH,
        trainable=False)
model.add(embedding_layer)
# conv_model.add(SpatialDropout1D())
model.add(Conv1D(NUM_FILTERS, KERNEL_SIZE, activation = 'relu',kernel_regularizer =tf.keras.regularizers.l2(l=0.09)))
model.add(Dropout(0.2))
# conv_model.add(BatchNormalization())
model.add(MaxPool1D(pool_size = 12))

model.add(LSTM(LSTM_HIDDEN_DIM_SIZE, activation = 'relu', dropout=rate_drop_lstm, recurrent_dropout=rate_drop_lstm))

# conv_model.add(MaxPool1D(pool_size = 15))

# conv_model.add(GlobalMaxPool1D())
# conv_model.add(Flatten())
model.add(Dense(2,activation='sigmoid'))
# lstm_model.add(LSTM(LSTM_HIDDEN_DIM_SIZE, activation = 'tanh', dropout=rate_drop_lstm, recurrent_dropout=rate_drop_lstm,return_sequences = True))
# lstm_model.add(LSTM(LSTM_HIDDEN_DIM_SIZE//2, activation = 'tanh', dropout=rate_drop_lstm, recurrent_dropout=rate_drop_lstm))
# lstm_model.add(Dense(LSTM_HIDDEN_DIM_SIZE//4, activation = 'relu'))
# lstm_model.add(LSTM(LSTM_HIDDEN_DIM_SIZE//4, activation = 'tanh',dropout=rate_drop_lstm, recurrent_dropout=rate_drop_lstm))


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [15]:
del conv_model

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 434, 300)          900132300 
_________________________________________________________________
conv1d (Conv1D)              (None, 423, 14)           50414     
_________________________________________________________________
dropout (Dropout)            (None, 423, 14)           0         
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 35, 14)            0         
_________________________________________________________________
unified_lstm (UnifiedLSTM)   (None, 5)                 400       
_________________________________________________________________
dense (Dense)                (None, 2)                 12        
Total params: 900,183,126
Trainable params: 50,826
Non-trainable params: 900,132,300
_____________________________________

In [22]:
model.reset_states()

In [15]:
# opt = tk.optimizers.Adam(learning_rate=0.005)

In [10]:
model.compile(loss='binary_crossentropy',
        optimizer='adam',
        metrics=['acc'])
history = model.fit(data_train,labels_train,epochs = 25,batch_size = 256, validation_data = [data_val,labels_val],shuffle=True)

Train on 7380 samples, validate on 821 samples
Epoch 1/25
7380/7380 [==============================] - 19s 3ms/sample - loss: 1.5996 - acc: 0.5106 - val_loss: 0.8030 - val_acc: 0.5670
Epoch 2/25
7380/7380 [==============================] - 16s 2ms/sample - loss: 0.7188 - acc: 0.5510 - val_loss: 0.6934 - val_acc: 0.5597
Epoch 3/25
7380/7380 [==============================] - 15s 2ms/sample - loss: 0.6910 - acc: 0.5543 - val_loss: 0.6881 - val_acc: 0.5597
Epoch 4/25
7380/7380 [==============================] - 16s 2ms/sample - loss: 0.6867 - acc: 0.5632 - val_loss: 0.6835 - val_acc: 0.5633
Epoch 5/25
7380/7380 [==============================] - 16s 2ms/sample - loss: 0.6831 - acc: 0.5716 - val_loss: 0.6831 - val_acc: 0.5621
Epoch 6/25
7380/7380 [==============================] - 15s 2ms/sample - loss: 0.6807 - acc: 0.5784 - val_loss: 0.6842 - val_acc: 0.5688
Epoch 7/25
7380/7380 [==============================] - 15s 2ms/sample - loss: 0.6805 - acc: 0.5853 - val_loss: 0.6845 - val_acc: 0

In [19]:
history = model.fit(data_train,labels_train,epochs = 10,batch_size = 256, validation_data = [data_val,labels_val],shuffle=True)

Train on 7380 samples, validate on 821 samples
Epoch 1/10
7380/7380 [==============================] - 9s 1ms/sample - loss: 0.3685 - acc: 0.8527 - val_loss: 0.5241 - val_acc: 0.9963
Epoch 2/10
7380/7380 [==============================] - 10s 1ms/sample - loss: 0.3745 - acc: 0.8496 - val_loss: 0.5676 - val_acc: 0.9799
Epoch 3/10
7380/7380 [==============================] - 9s 1ms/sample - loss: 0.4469 - acc: 0.8367 - val_loss: 0.6472 - val_acc: 0.9933
Epoch 4/10
7380/7380 [==============================] - 9s 1ms/sample - loss: 0.4571 - acc: 0.8520 - val_loss: 0.5933 - val_acc: 0.9909
Epoch 5/10
7380/7380 [==============================] - 9s 1ms/sample - loss: 0.4452 - acc: 0.8474 - val_loss: 0.7497 - val_acc: 0.9543
Epoch 6/10
7380/7380 [==============================] - 9s 1ms/sample - loss: 0.6233 - acc: 0.8407 - val_loss: 0.7647 - val_acc: 0.9945
Epoch 7/10
7380/7380 [==============================] - 9s 1ms/sample - loss: 0.7763 - acc: 0.8370 - val_loss: 1.0445 - val_acc: 0.9671


In [51]:
model.save("./Checkpoints/conv_model.h5")

In [52]:
model.save_weights("./Checkpoints/conv_weights")

In [12]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

pred = model.predict(x_test)
pred = [np.argmax(x) for x in pred]

print(f"F1-score : {f1_score([np.argmax(x) for x in y_test],pred,pos_label=1)}")
print(f"Accuracy-score : {accuracy_score([np.argmax(x) for x in y_test],pred)}")

print(f"Precision : {precision_score([np.argmax(x) for x in y_test],pred,pos_label = 1)}")
print(f"Recall  : {recall_score([np.argmax(x) for x in y_test],pred,pos_label = 1)}")

F1-score : 0.9961575408261287
Accuracy-score : 0.9960994636762555
Precision : 0.9952015355086372
Recall  : 0.9971153846153846


In [21]:
from sklearn.metrics import accuracy_score

pred = conv_model.predict(x_test)
pred = [np.argmax(x) for x in pred]



In [23]:
from sklearn.metrics import f1_score
f1_score([np.argmax(x) for x in y_test],pred,pos_label=1)

0.9956751561749159

In [24]:
accuracy_score([np.argmax(x) for x in y_test],pred)

0.9956118966357874

In [25]:
from sklearn.metrics import roc_auc_score
roc_auc_score([np.argmax(x) for x in y_test],pred)

0.9956041238682188

In [19]:
i=0
for seq in data_train[128:250]:
    for el in seq:
        if np.isnan(el):
            print(i)
                
#         if type(arr) is np.ndarray:
#             print(i)
    i+=1

In [2]:
import pickle
def save_pickle_file(file,file_path,protocol=None):
    with open(file_path,"wb") as f:
        if protocol:
            pickle.dump(file,f,protocol = protocol)
        else:
            pickle.dump(file,f)

def load_pickle_file(file_path, protocol=None):
     with open(file_path,"rb") as f:
        if protocol:
            return pickle.load(f)
        else:
            return pickle.load(f)
